In [1]:
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import seaborn as sns
import random
%matplotlib inline 

In [2]:
df = pd.read_csv("kenya_data/diaries_transactions_all.csv", dtype={'account_startclose_balance': str})

# 1. Truncating and selecting the relevant loans.

See file "Pure_Truncation_Start_Bal_Ella.ipynb" in the folder datasets.

## 1. 1. Loan categories

We decided to only use loans 
1. "Individual Business or Agricultural loans" in the family "formal loan", 
2. "Consumer/personal loan" in the family "formal loan",
3. "Joint liability loan" in the family "formal loan",
4. "Borrowing from an informal group" in the family "informal group".
The reason we chose these loans is that they look similar to a "classical loan": First borrowing some money and then payments over a certain interval.

This means that we did not use loans that were very rare like student loans, or school fee loans, and loans that did not fit well into this format, like wage advances, credit cards, informal credit at a store, borrowing from family and friends and moneylender or shylock borrowing.

## 1.2. Restrictions

We excluded loans that have unexplained balance adjustments, a starting balance of less than 10 KES and several new borrowings on the same day.

## 1.3 Truncation

Eventhough we only selected the loans most similar to a classical loans, many of them have new borrowings before or after an old borrowing is paid off. To be able to apply the metric we chose to all loans, we split the loans into several chunk, each chunk starting with a new borrowing. If there was a new borrowing and other transactions on the same day, we assigned all transactions on the same day of the new borrowing to the next chunk.

## 1.4 Starting balance and running balance

For each chunk we add a column "start_bal" and "bal_evol" that contain the starting balance of the chunk and the debt still owed after each transaction.

We then remove all loans with starting balance less than or equal to 10 KES (3.7%).

## 1.5 Issues

Right now the starting balance and running balance are computed for each chunk separately. It might be better to first compute the running balance for each account and then separate into chunks because sometime people paid in before taking the loan and sometimes they already had debt when taking a new loan.

# 2. Assigning a score to the loans

## 2.1 Checking if loan was paid back in a certain amount of days (e.g. 150) 

The file `steady_payments_or_not-pureTruncElla.ipynb` in `julia` folder was used to figure out if a loan was paid back in a certain amount of days. The function `find_borrowers_paid_back_in_x_days` takes in a transaction dataframe and threshold number of days, and will spit out whether the balance at the last day (of the truncated loan) is 0 or not.

## 2.2 Checking if a loan was paid back steadily in a certain threshold of days (e.g. every 33 days) 

The file `steady_payments_or_not-pureTruncElla.ipynb` in `julia` folder also has a function called `find_borrowers_num_days_threshold` that takes in a transaction dataframe, the output of `find_borrowers_paid_back_in_x_days`, a threshold (e.g. 33), and x_days (e.g. 150) that we want to observe the loan.

This function then calculates the time between each "payment" transaction. If the loan was paid back in x_days, it just looks at this vector. If it was not paid back in x_days, it adds a "difference" metric that takes the starting_day+x_days-ending_day and adds this number to our vector; which should be more than x_days if someone stopped paying the loan.

The function `create_outcomes` then creates a csv file with outcomes for each loan

# 3. Assigning a score to a person

## 3.1 Categorizing a loanpayer by their loan scores from (2)

In `categorize_loanpayer_per_person` in /julia/, I use the scores output in (2) to assign someone an overall loanpayer score. If they met either criteria (paid back loan quickly within threshold, eg 150 days or paid back loan regularly eg within 33 days for each payment) then that loan was scored with score 1, else it was scored 0. 

I then took the mean of 0s and 1s per person, and thresholded at <= 0.5 for bad loanpayers, and >0.5 for good loanpayers. This made a binary score on a per-person basis

# 4. Preparing the other variables

Generally we want to generate time independent variables for individuals for each dataset we have, so that we can use them for modeling. Some datasets are in terms of households, in that case we would add all variables associated to a household to each individual that lives in a household.

## 4.1. Well-being

There seems to have been a problem when the data was saved, as the number of unique individual ids is 16. So instead I computed an average happiness, confidence, relationships and economical well-being score for each household, that will then be assigned to each individual in that household.

file : datasets/Well-Being_CSV.ipynb

## 4.2. Goings-on

Again the average number of each incident is computed for each household.

file: datasets/Goings-On-To-csv.ipynb

## 4.3. Income

For each person the average, standard deviation, min and max income over all months is contained. Also we have the average, median and standard deviation of income in several different categories, over all months. Finally the main source of income for each individual is contained. Here we can just use the original dataset.

inc_ind_over_trx_yr_mo_kes.csv

## 4.4 Consumption

For each household, there is a summary of their consumption habits over time ... all I had to do was map that to individual ids.

file: consumption_mapped_to_m_ids.csv

## 4.5 Education

There were education stats per person and per household; I merged them on HH id and that propagated to individual ids. 

file: edu_hh_ind_merge.csv

## 4.6 Demographics

### Issue:
We are not sure if we should include or exclude resources received from family and friend from these datasets.

### 4.7. Poverty 

We can evaluate how much money left at the end of the month: we are provided with information about average monthy income and consumption. Some families are left with negative balance even with the resouces received from friends and family. At the intial stage it is a good indicator whether the family member of particular hh will be able to pay off the debt in case if they have some money left at the end of the month (paycheck cycle). 
Called file as inc-cons. 

### 4.8 Housing 

Good indicator for the case of a bigger loan evaluation. Or in case if the person will have a lower credit score (acording to "steady payment behaviour" criteria) he an pledge the house to the bank and take bigger loan.  
Many hh have several properties, and they are able to use them for agrculture or rent. 

### 4.9 Remittances 
data about resources given and received. Not quite sure if we need to use this. 